In [1]:
#importanción de pandas
import pandas as pd


In [2]:
#lectura del e importación del csv como DataFrame
df = pd.read_csv("./input/GSAF5.csv")

In [3]:
df.shape

(5992, 24)

In [4]:
print(df.columns)

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')


In [5]:
#voy a renombrar columnas para facilitar el acceso
df.columns =['caseNumber', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal(Y/N)', 'time',
       'species', 'investigatorSource', 'pdf', 'hrefFormula', 'href',
       'caseNumber1', 'caseNumber2', 'originalOrder', 'Unnamed22',
       'Unnamed23']
df.head()

,caseNumber,date,year,type,country,area,location,activity,name,sex,...,species,investigatorSource,pdf,hrefFormula,href,caseNumber1,caseNumber2,originalOrder,Unnamed22,Unnamed23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [6]:
#Compruebo columnas con valores nulos
null_cols = df.isnull().sum()
null_cols[null_cols > 0]
null_cols

caseNumber               0
date                     0
year                     0
type                     0
country                 43
area                   402
location               496
activity               527
name                   200
sex                    567
age                   2681
injury                  27
fatal(Y/N)              19
time                  3213
species               2934
investigatorSource      15
pdf                      0
hrefFormula              1
href                     3
caseNumber1              0
caseNumber2              0
originalOrder            0
Unnamed22             5991
Unnamed23             5990
dtype: int64

In [7]:
#Guardo un data Frame con las columnas que no tienen valores nulos, están limpias
dfNotNull = df[["caseNumber","date","year","type","pdf","caseNumber1","caseNumber2","originalOrder"]]

In [8]:
'''En este punto tomo la decisión de dropear del Data Frame principal las columnas Unnamed22 y 23,
pues la totalidad de sus registros prácticamente son nulos'''
df = df.drop(columns = ["Unnamed22","Unnamed23"])

In [9]:
'''la siguiente columnas con bastantes nulos es edad, esta vez lo que voy a hacer es meter en el dfNotNull una columna
 que se llame age(-1NoData) con los valores validos mas -1 para los valores inválidos'''
dfNotNull["age(-1NoData)"] = df['age'].fillna(-1)

In [10]:
'''La siguiente columna con muchos nulos, más de la mitad es time, voy a hacer un display para decidir si me interesa
guardar esa info'''

pd.DataFrame(df['time'].value_counts())

,time
Afternoon,174
11h00,123
Morning,107
12h00,107
15h00,100
...,...
10h07,1
12h35,1
10h30 or 13h30,1
2 hours after Opperman,1


In [11]:
'''Voy a cambiar los nulos, merece la pena limpiar la columna'''
dfNotNull["timeDirty"] = df['time'].fillna("noData")

In [12]:
'''Voy a analizar species'''
pd.DataFrame(df["species"].value_counts())

,species
White shark,161
Shark involvement not confirmed,80
Tiger shark,68
Bull shark,52
6' shark,40
...,...
Possibly a 10' bull shark,1
"""Shark caught later""",1
"White shark, 1.8 m to 2.1 m [6' to 7'] according to Shreuder and a witness",1
"Grey nurse shark, 4'",1


In [13]:
'''Decido cambiar los nulos a Shark involvement not confirmed, parece lógico agrupar en esta categoría los nulos'''
dfNotNull["species"] = df['species'].fillna("Shark involvement not confirmed")


In [14]:
'''el resto de columnas que tienen no válidos contienen pocos por lo que voy a meter valores que no lleven a confusión'''

'el resto de columnas que tienen no válidos contienen pocos por lo que voy a meter valores que no lleven a confusión'

In [15]:
dfNotNull["country"] = df['country'].fillna("unknown")
dfNotNull["area"] = df['area'].fillna("unknown")
dfNotNull["location"] = df['location'].fillna("unknown")
dfNotNull["activity"] = df['activity'].fillna("unknown")
dfNotNull["name"] = df['name'].fillna("unknown")
dfNotNull["sex"] = df['sex'].fillna("unknown")
dfNotNull["injury"] = df['injury'].fillna("unknown")
dfNotNull["fatal(Y/N/unknow)"] = df['fatal(Y/N)'].fillna("unknown")
dfNotNull["investigatorSource"] = df['investigatorSource'].fillna("unknown")
dfNotNull["hrefFormula"] = df['hrefFormula'].fillna("unknown")
dfNotNull["href"] = df['href'].fillna("unknown")
#Compruebo que no hay nulos
null_cols = dfNotNull.isnull().sum()
#null_cols[null_cols > 0]
null_cols


caseNumber            0
date                  0
year                  0
type                  0
pdf                   0
caseNumber1           0
caseNumber2           0
originalOrder         0
age(-1NoData)         0
timeDirty             0
species               0
country               0
area                  0
location              0
activity              0
name                  0
sex                   0
injury                0
fatal(Y/N/unknow)     0
investigatorSource    0
hrefFormula           0
href                  0
dtype: int64

In [16]:
#limpieza de "fatal"
fatal = "fatal(Y/N/unknow)"
cleanFatal = dfNotNull[fatal]
cleanFatal.value_counts()

N          4315
Y          1552
UNKNOWN      94
unknown      19
 N            8
#VALUE!       1
F             1
N             1
n             1
Name: fatal(Y/N/unknow), dtype: int64

In [17]:
from functions import cleanTextInFatal
cleanFatal = cleanFatal.apply(cleanTextInFatal)
cleanFatal.value_counts()


N          4421
Y          1552
UNKNOWN      19
Name: fatal(Y/N/unknow), dtype: int64

In [18]:
#Lo añado a un dataFrame que tendrá los datos limpios
dfClean = pd.DataFrame(cleanFatal)
dfNotNull = dfNotNull.drop(columns =fatal)
#elimino los objetos que no voy a usar
del cleanFatal
del fatal

In [19]:
dfNotNull.columns

Index(['caseNumber', 'date', 'year', 'type', 'pdf', 'caseNumber1',
       'caseNumber2', 'originalOrder', 'age(-1NoData)', 'timeDirty', 'species',
       'country', 'area', 'location', 'activity', 'name', 'sex', 'injury',
       'investigatorSource', 'hrefFormula', 'href'],
      dtype='object')

In [20]:
#voy a analizar type
dfNotNull["type"].value_counts()

Unprovoked      4386
Provoked         557
Invalid          519
Sea Disaster     220
Boat             200
Boating          110
Name: type, dtype: int64

In [21]:
#diría que no tiene demasiado sentido Sea disaster, boat y boating, los voy a cambiar a junto con Ivalid a unknown
dfClean["type"]=dfNotNull["type"].replace(
    ['Invalid', 'Sea Disaster',"Boat","Boating"],
    'UNKNOWN',
    
)
dfNotNull=dfNotNull.drop(columns = "type")
dfClean["type"].value_counts()

Unprovoked    4386
UNKNOWN       1049
Provoked       557
Name: type, dtype: int64

In [22]:
dfNotNull.columns

Index(['caseNumber', 'date', 'year', 'pdf', 'caseNumber1', 'caseNumber2',
       'originalOrder', 'age(-1NoData)', 'timeDirty', 'species', 'country',
       'area', 'location', 'activity', 'name', 'sex', 'injury',
       'investigatorSource', 'hrefFormula', 'href'],
      dtype='object')

In [23]:
#paso a activity
activitySerie = dfNotNull["activity"]
activitySerie.value_counts().head(40)


Surfing                 904
Swimming                819
unknown                 527
Fishing                 414
Spearfishing            321
Bathing                 153
Wading                  144
Diving                  123
Standing                 97
Snorkeling               77
Scuba diving             74
Body boarding            54
Body surfing             48
Swimming                 47
Pearl diving             32
Treading water           32
Boogie boarding          28
Kayaking                 28
Free diving              26
Fell overboard           21
Windsurfing              18
Boogie Boarding          16
Shark fishing            15
Walking                  15
Surf skiing              12
Canoeing                 12
Rowing                   12
Fishing                  12
Surf-skiing              12
Floating                 12
Fishing for sharks       11
Surf fishing             11
Freediving               10
Kayak Fishing            10
Sponge diving            10
Sitting on surfboard

In [24]:
"Voy a hacer alguno replace para unificar algunos datos fáciles"
activitySerie=activitySerie.replace(
    ['Swimming ',"Floating on his back"],
    'Swimming',
    
)
activitySerie=activitySerie.replace(
    'Fishing ',
    'Fishing',
    
)
activitySerie=activitySerie.replace(
    ['Diving for abalone',"Diving for trochus","Fishing for mackerel","Free diving for abalone","Sponge diving",
    "Hard hat diving", "Diving for coins","Freediving"
],
"Diving"    
)

activitySerie=activitySerie.replace(
    ['Surfing (sitting on his board)',"Sitting on surfboard"],
"Surfing"   
)

activitySerie=activitySerie.replace(
    ['Treading water',"Playing","Walking","Standing"],
"Wading"  
)
activitySerie=activitySerie.replace(
    ['Spearfishing ',"Skindiving","Spearfishing on Scuba","Scuba diving (but on surface)","Scuba diving (submerged)"],
"Spearfishing"   
)
activitySerie=activitySerie.replace(
    ["Murder","Sea Disaster"],
"unknown"  
)
activitySerie=activitySerie.replace(
    ['Fell overboard ',"Canoeing","Fell into the water","Boat","Dangling feet in the water","Sailing "],
"Boating"   
)

activitySerie=activitySerie.replace(
    ['Surf-skiing  ',"Surf skiing "],
"Surf skiing"   
)

activitySerie=activitySerie.replace(
    ['Sea Disaster '],
"Sea Disaster"   
)
 
activitySerie.value_counts().head(40)


Surfing                  920
Swimming                 873
unknown                  539
Fishing                  426
Spearfishing             349
Wading                   295
Diving                   181
Bathing                  153
Snorkeling                77
Scuba diving              74
Body boarding             54
Body surfing              48
Boating                   38
Pearl diving              32
Kayaking                  28
Boogie boarding           28
Free diving               26
Fell overboard            21
Surf skiing               18
Windsurfing               18
Boogie Boarding           16
Shark fishing             15
Rowing                    12
Surf-skiing               12
Floating                  12
Surf fishing              11
Fishing for sharks        11
Kayak Fishing             10
Sailing                    9
Sea disaster               8
Paddle boarding            7
Kite Surfing               6
Fishing on a boat          5
Freedom swimming           5
Shipwreck     

In [25]:
#Voy a categorizar la columna en los grupos que actualmente tienen más de 6 registros, 
#para ello hago una lista con los que tienen menos de 11 para ir filtrando y ver en qué categoria los meto 

badGroupsInActivity =pd.DataFrame(activitySerie.value_counts()).query("activity<11").index

badGroupsInActivity = [str(ele) for ele in badGroupsInActivity]
print(len(badGroupsInActivity))

1436


In [26]:
#Programé unas funciones para que me formatearan en las categorías la columna
from functions import cleanActivity


In [27]:
activitySerie = activitySerie.apply(cleanActivity,lista = badGroupsInActivity)

activitySerie.value_counts().head(40)


Bathing               1019
Surfing               1015
Swimming               873
Fishing                650
unknown                539
Spearfishing           399
Diving                 378
Wading                 295
Sailing                139
Scuba diving           114
Snorkeling              77
Body boarding           54
Shark fishing           51
Body surfing            48
Boating                 38
Pearl diving            32
Kayaking                28
Boogie boarding         28
Free diving             26
Surf fishing            24
Surf skiing             23
Fell overboard          21
Windsurfing             21
Boogie Boarding         16
Body surf               16
Floating                12
Rowing                  12
Surf-skiing             12
Fishing for sharks      11
Name: activity, dtype: int64

In [28]:
"Vuelvo a hacer print de la lista de valores que no quiero por si me hubiera dejado algo"
badGroupsInActivity =pd.DataFrame(activitySerie.value_counts()).query("activity<11").index

badGroupsInActivity = [str(ele) for ele in badGroupsInActivity]
print(len(badGroupsInActivity))

0


In [29]:
#perfecto, aún veo un par de índices que puedo categorizar, puedo usar la misma función para ello

In [30]:
listaConCategoriasFusionables =["Surf-skiing","Boating","Fishing for sharks"]
listaConCategoriasFusionables = [str(ele) for ele in listaConCategoriasFusionables]
activitySerie=activitySerie.replace(
    "Surf-skiing",
"Surf skiing"  
)
activitySerie=activitySerie.replace(
    ["Rowing","Boating"],
"Sailing"  
)
activitySerie=activitySerie.replace(
    'Fishing for sharks',
"Shark fishing"  
)

In [31]:
pd.DataFrame(activitySerie.value_counts())

,activity
Bathing,1019
Surfing,1015
Swimming,873
Fishing,650
unknown,539
Spearfishing,399
Diving,378
Wading,295
Sailing,189
Scuba diving,114


In [32]:
#En este punto creo que activity está muy limpio, lo añado y borro variables que no vaya a usar
dfClean["activity"] = activitySerie
del activitySerie
del badGroupsInActivity
dfNotNull = dfNotNull.drop(columns = "activity")

In [33]:
#Voy a ver si href y hrefFormula es información duplicada
import numpy as np
array = (dfNotNull["href"].values == dfNotNull["hrefFormula"].values)
array.size - np.sum(array)

54

In [34]:
#De lo que se deduce que solo hay 54 valores que no coinciden, me voy a quedar son sus índices
indicesHref = pd.DataFrame(np.where(array == False)).T.astype("int")[0].tolist()

In [35]:
#Programé una función para comprobar si la url que se le pasa es válida y el archivo exite, si lo es devuelve la Url, si no Unknown
from functions import checkUrlValid


In [36]:
"""#ok ya tengo las válidas de href, ahora solo voy a pasar la función en href formula para los índice unknown por si puedo salvar alguna
validHref = dfNotNull["href"].apply(checkUrlValid)
validHref"""

'#ok ya tengo las válidas de href, ahora solo voy a pasar la función en href formula para los índice unknown por si puedo salvar alguna\nvalidHref = dfNotNull["href"].apply(checkUrlValid)\nvalidHref'

In [37]:
"""#como son solo 4 índices los pongo a mano
validURL["href"][listaUnknown[0]]
checkUrlValid(str(dfNotNull["hrefFormula"][listaUnknown[0]]))"""

'#como son solo 4 índices los pongo a mano\nvalidURL["href"][listaUnknown[0]]\ncheckUrlValid(str(dfNotNull["hrefFormula"][listaUnknown[0]]))'

In [54]:
dfNotNull["country"].value_counts().head(70)

USA                 2116
AUSTRALIA           1279
SOUTH AFRICA         565
PAPUA NEW GUINEA     133
NEW ZEALAND          125
                    ... 
SAMOA                  7
SEYCHELLES             7
THAILAND               6
OKINAWA                6
SINGAPORE              6
Name: country, Length: 70, dtype: int64

In [57]:
#Pienso que àises está bastande limpio así que lo voy a añadir tal cual
dfClean["country"] = dfNotNull["country"]
dfNotNull=dfNotNull.drop(columns="country")

In [91]:
#Voy a analizar la columna de sex, parece facil
dfNotNull["sex"].value_counts().index.tolist()


['M', 'F', 'unknown', 'M ', '.', 'lli', 'N']

In [93]:
dfNotNull["sex"] = dfNotNull["sex"].replace(["M "],"M")
dfNotNull["sex"] = dfNotNull["sex"].replace([".","lli","N"],"unknown")
dfClean["sex"]= dfNotNull["sex"]
dfNotNull =dfNotNull.drop(columns = "sex") 


0       M
1       M
2       M
3       M
4       M
       ..
5987    M
5988    M
5989    M
5990    M
5991    M
Name: sex, Length: 5992, dtype: object

In [94]:
dfClean["sex"].value_counts().index.tolist()

['M', 'F', 'unknown']